# Lesson 31: Introduction to Biopython

*This lesson was generated from a Jupyter notebook.  You can download the notebook [here](l31_intro_to_biopython.ipynb).*

In [2]:
# Stuff from standard library
import random
import time
import timeit

# Import Biopython modules
import Bio
import Bio.Seq
import Bio.Alphabet.IUPAC
import Bio.SeqRecord
import Bio.SeqIO
import Bio.AlignIO
import Bio.Entrez

## Sequence Data and Biology
In Biology today, there are broadly two types of data: sequence data and "everything else." So far, our discussion has focused on the "everything else," so this tutorial will introduce sequence data and how to mangle it in useful ways to answer biological questions.

As a quick refresher, here are some basics:

Sequences in biology come in two forms: nucleotide and protein. Most of the public databases are composed of experimentally observed nucleotide sequences, i.e. from DNA "sequencers." Most protein sequences in the databases have been deduced from observed nucleotide sequences, but a tiny minority have directly been observed from experimental methods like [Edman degradation](https://en.wikipedia.org/wiki/Edman_degradation). We go from nucleotide sequences to proteins through a genetic code, typically written as a [codon table](https://en.wikipedia.org/wiki/DNA_codon_table).

## Modules in Python
A number of modules have been written for Python to deal with sequence data including [Biopython](http://biopython.org/) and [scikit-bio](http://scikit-bio.org/).

Biopython has a large, mature codebase and extensive functionality as well as a lively [listserv](http://biopython.org/wiki/Mailing_lists) to exchange questions and answers. There are a number of [books on Biopython](http://www.amazon.com/s?ie=UTF8&field-keywords=Biopython), question-answer archives on [Stackoverflow](http://stackoverflow.com/search?q=biopython) and [Biostars](https://www.biostars.org/), and a website and [tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html).

Scikit-bio is still in beta. I don't know too much about it since Biopython does everything I need.

This tutorial will cover Biopython.

In [3]:
#check what version you have installed
print(Bio.__version__)

1.65


If you installed Biopython through `conda install`, you will be running version `1.65`.

## The Sequence Object

Let's start by exploring basic sequence objects:

In [4]:
# Instantiate sequence object
myseq = Bio.Seq.Seq('CAT CAT CAT UTC UUG UCG CTU CCU CTU GUG UUG TTC UTG')

# Look at it
myseq

Seq('CAT CAT CAT UTC UUG UCG CTU CCU CTU GUG UUG TTC UTG', Alphabet())

`myseq` is an object of the `Seq` class in Biopython. Observing the output above, it is really just a string associated with a certain `Alphabet`. Since we didn't specify one, the generic `Alphabet` was assigned

To be more precise, we can directly specify an alphabet using the [IUPAC convention](http://biopython.org/DIST/docs/api/Bio.Alphabet.IUPAC-module.html). 

In [5]:
# Directly specify alphabet convention
myseq = Bio.Seq.Seq(str(myseq), Bio.Alphabet.IUPAC.IUPACUnambiguousDNA())

myseq

Seq('CAT CAT CAT UTC UUG UCG CTU CCU CTU GUG UUG TTC UTG', IUPACUnambiguousDNA())

What's the difference between an Unambiguous code and an Ambiguous code?

In the course of seqencing, different sorts of errors can occur. These can vary from being unable to specify the base at a given position to being unable to differentiate between two of the four bases. To accomidate these, there is an [ambiguity code](http://www.bioinformatics.org/sms/iupac.html). For example, `N` represents any nucleotide, and `R` represents `A` or `G`.

It's useful to consider this when you delve into the dark depths of the public sequence databases.

Now, let's translate our sequence into protein using the `translate()` method of a `Seq` object.

In [6]:
myseq.translate()

TranslationError: Codon ' CA' is invalid

Careful! Biopython does not do error checking when a sequence object is created to make sure that the characters in the string are actually a subset of the specified alphabet, but translating spaces doesn't make much sense and raises an exception (of course!).

In many ways, `Bio.Seq.Seq` objects act as normal Python strings. Let's test a few operations:

In [7]:
print("Original              :", myseq)
print("First character       :", myseq[0])
print("Every fourth character:", myseq[::4])

myseq.replace(" ", '')

Original              : CAT CAT CAT UTC UUG UCG CTU CCU CTU GUG UUG TTC UTG
First character       : C
Every fourth character: CCCUUUCCCGUTU


AttributeError: 'Seq' object has no attribute 'replace'

`Bio.Seq.Seq` objects don't have a replace function, so let's turn our `Bio.Seq.Seq` object into a string, use Python's native string replacement to get rid of the spaces, recreate our `Bio.Seq.Seq` object, and then translate!

In [8]:
# Convert to string and replace spaces with nothing
myseq = str(myseq).replace(' ', '')

# Reinstantiate the Seq object
myseq = Bio.Seq.Seq(myseq, Bio.Alphabet.IUPAC.IUPACUnambiguousDNA())

# Now try translating it into protein
proteinseq = myseq.translate()

# Check out result
proteinseq

Seq('HHHFLSLPLVLFL', ExtendedIUPACProtein())

It's a "random" protein sequence when translated. How about the reverse complement? `Bio.Seq.Seq` objects have a method that performs this for us.  (Don't hate us for making you write functions to make reverse complements.  It's good practice!)

In [9]:
# Make reverse complement
myseq = myseq.reverse_complement()

# Translated it into protein
proteinseq = myseq.translate()

print(myseq)
print(proteinseq)

CAUGAACUUCUCUAGUGGUAGCGUCUUGAUATGATGATG
HELL*W*RLDMMM


How about if we remove the trailing `M`s to get closer to `Hello World`? Remember we can use string-like splicing

In [10]:
print("original       :", proteinseq)
print("No trailing M's:", proteinseq[:-3])

original       : HELL*W*RLDMMM
No trailing M's: HELL*W*RLD


And here you have it, your first (almost `Hello World`) protein sequence. Scrolling back, you may notice that the stop codons are encoded by `UAG`.

In some methanogenic archaea, `UAG` doesn't encode stop but instead [pyrrolysine](https://en.wikipedia.org/wiki/Pyrrolysine), whose short codes are `Pyr` and `O`.

The NCBI maintains the authoritative list of genetic codes and their numbers [here](http://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi). The `translate()` method  defaults to "`1. Standard Genetic Code`." Check out the [docs](http://biopython.org/DIST/docs/api/Bio.Seq-module.html#translate) to verify this yourself.


## The Sequence record and File I/O

While raw sequences are useful, in the context of a dataset we need to have a way to store all the metadata associated with a single sequence. Biopython's `SeqRecord` does this job.

In [11]:
myseqrecord = Bio.SeqRecord.SeqRecord(
                       id = "MyFirstSeq",
                       description = "This is my first sequence",
                       seq = myseq)

print(myseqrecord)

ID: MyFirstSeq
Name: <unknown name>
Description: This is my first sequence
Number of features: 0
Seq('CAUGAACUUCUCUAGUGGUAGCGUCUUGAUATGATGATG', IUPACUnambiguousDNA())


Sequences are stored in various formats. Biopython can handle [many of them](http://biopython.org/wiki/SeqIO#File_Formats). The simplest is the [FASTA format](https://en.wikipedia.org/wiki/FASTA_format), which consists of a header/descrpition line preceeded by a '>' and then the sequence, typically wrapped in 60 or 80 character lines.

Another common format is the [GenBank Flat File](http://www.ncbi.nlm.nih.gov/Sitemap/samplerecord.html) format ("GenBank" for short), which includes the sequence along with metadata (typically a significant amount) in an structured, parseable format. We will discuss this in more depth in a moment.

Another, less common format is [SeqXML](http://seqxml.org/xml/Main.html), which formats sequence information into a rigid framework that is less human readable but more easily machine parseable. Read more about XML [here](https://en.wikipedia.org/wiki/XML).

In [12]:
print(myseqrecord.format("fasta"))

>MyFirstSeq This is my first sequence
CAUGAACUUCUCUAGUGGUAGCGUCUUGAUATGATGATG



In [13]:
print(myseqrecord.format("genbank"))

LOCUS       MyFirstSeq                39 bp    DNA              UNK 01-JAN-1980
DEFINITION  This is my first sequence
ACCESSION   MyFirstSeq
VERSION     MyFirstSeq
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
ORIGIN
        1 caugaacuuc ucuaguggua gcgucuugau atgatgatg
//



In [14]:
print(myseqrecord.format("seqxml"))

<?xml version="1.0" encoding="utf-8"?>
<seqXML xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.seqxml.org/0.4/seqxml.xsd" seqXMLversion="0.4"><entry id="MyFirstSeq"><description>This is my first sequence</description><DNAseq>CAUGAACUUCUCUAGUGGUAGCGUCUUGAUATGATGATG</DNAseq></entry></seqXML>


Let's generate a set of random sequences, write them to file, read them back in, and then select a subset.  Since this is just an illustration, we're going to do a quick-and-dirty generation of random sequences; they'll all have the same number of A's, T's, G's, and C's, just scrambled.

In [15]:
# generate a list of 100 sequences
myseq = ['A','C','G','T'] * 250

# Initialize list of sequences
seq_record_list = []

# Make 100 random sequences and store
for i in range(100):
    # shuffle the list in place
    random.shuffle(myseq)
    
    #join the characters in the list since the Seq construtor needs a string
    myseq_object = Bio.Seq.Seq("".join(myseq), 
                               Bio.Alphabet.IUPAC.IUPACUnambiguousDNA)

    randomized_seq_record = Bio.SeqRecord.SeqRecord(
                                    id = str(i), 
                                    description = "",
                                    seq = myseq_object)

    seq_record_list.append(randomized_seq_record)
    
# Take a look
seq_record_list

[SeqRecord(seq=Seq('TCAACTATCACGGGTGCGTCACCAAGGCCTACGAAAGATCCTCTCTGCTAAGCG...TTT', <class 'Bio.Alphabet.IUPAC.IUPACUnambiguousDNA'>), id='0', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('CTCAAAAAAAGCCTGAATATATTATAACACATCGATTCAAGCACGGGTCGGCAC...CTG', <class 'Bio.Alphabet.IUPAC.IUPACUnambiguousDNA'>), id='1', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('CTGGTTCCGAAGACAATTAGCTGGCTCCCAAATTCTTCGGATTCCCCGGGACGC...CCG', <class 'Bio.Alphabet.IUPAC.IUPACUnambiguousDNA'>), id='2', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('AGTGAGCCGAGGGTCCGTGCCCACCTCCAGCTCGGCAAGCCGTAGAATTCAACA...ATG', <class 'Bio.Alphabet.IUPAC.IUPACUnambiguousDNA'>), id='3', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('CACTCCAGCTCTCTGGAAGGTTCCCTTACACCGAGTACGACCTATTCTTATCCG...GCG', <class 'Bio.Alphabet.IUPAC.IUPACUnambiguousDNA'>), id='4', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('CCGAGCCAAC

Now we have a list of sequence records. Let's write them to file in FASTA format since we don't have any information for each other than the id.

To do this, we should use a method from the [`Bio.SeqIO` module](http://biopython.org/DIST/docs/api/Bio.SeqIO-module.html).

Typically, the following file extension conventions are used:

| Extension | Meaning         |
|-----------|-----------------|
|   ffn     | FASTA nucleotide|
|   fna     | FASTA nucleotide|
|   faa     | FASTA protein   |
|  fasta    | FASTA generic   |
|   gb      | GenBank         |

While file extensions don't affect data stored in the file, keeping to convention will help maintain everyone's sanity!

In [16]:
count = Bio.SeqIO.write(seq_record_list, "l31_random_seqs.fna", "fasta")
print("Number of records written:", count)

Number of records written: 100


Take a look at the output file `l31_random_seqs.fna` in your favorite text editor or pager.

In [17]:
!cat l31_random_seqs.fna

>0
TCAACTATCACGGGTGCGTCACCAAGGCCTACGAAAGATCCTCTCTGCTAAGCGGGCTTT
TCATTCAAGATGTCGATGACCGTTTCATTTCAGCGTAATCCAGCATGCTTATGTGTGTAG
GTCTATGGGGCAGCAGACATTGGCTCGTCAACGAGGGGGACTGGGTAAGTCCCTACTCTC
TTGCGCCTACATTTGCTAAGCTTTCGTTTGGATAGTTGAATAAAAACTAAAATGGGGCAG
AGCTTCGGCTAGTAGCAGGCGCCTTGATCAAGGTGCTGGATTTTAAAAATTTCGCGGGTC
CTAGGTGTGAGCCTTGGCGATTAGTTTACAACACGGCGTTCACAACCGATTGATACTTGA
GCCACTTGGTCAGGAATTACCAGCCCTTCACACCAGTTCCTCGACGTAGAACTAGCAGAG
CGAAGATTTCTGTCTCGAAGTGATATTGCGGTATCATTTTGGCACGTACTGTAGCTAGCG
GAGTTGCGCGAAGCGACTAGCATATACTTATCGGCGCGCTCCTGTAGCCTAAAAATGTTT
CTCAAGCCTACAGACCATTCCGAAGACAAGCTCGCAGCTAACTAGGGCCGGAAGACAAGA
CGAGCCTGAGGTAACGGTCTGCCAGTTTGGAATCAACGCTACGTGAGAAAGCGACTACCA
ACCGATTGGTTTCCCTTTTTGTCTTCGACTATGTCGGATTGACTGTTAGAGGTGCTACTT
GGAGTTGCTCATGTCCGGAACGGTCACGAAGAAGTCCTAAACGACCCTTGGGCTGTGTGT
ACAGCTCACCACGAAGAGATGTCGTCGTAGATATGCAACCCGGAACCTACCAGTACCCGT
AAACACTGCTGCCGAGCCCGGGTATGGCGCACCGCCCCCCTGGGGCTATTTCATGCCTTA
ATAGGCCCCAACTTCATAATAACTTTCGAAGAAGCTCAAAGCGCTATGCCAGGCAGTAGC
TGCCACGTGAGCCCAAAATAG

One way to read in sequences is to iterate through the records.  `Bio.SeqIO.parse` handles the parsing for you!

In [18]:
for record in Bio.SeqIO.parse("l31_random_seqs.fna", "fasta"):
    print(record.format("fasta"))

>0
TCAACTATCACGGGTGCGTCACCAAGGCCTACGAAAGATCCTCTCTGCTAAGCGGGCTTT
TCATTCAAGATGTCGATGACCGTTTCATTTCAGCGTAATCCAGCATGCTTATGTGTGTAG
GTCTATGGGGCAGCAGACATTGGCTCGTCAACGAGGGGGACTGGGTAAGTCCCTACTCTC
TTGCGCCTACATTTGCTAAGCTTTCGTTTGGATAGTTGAATAAAAACTAAAATGGGGCAG
AGCTTCGGCTAGTAGCAGGCGCCTTGATCAAGGTGCTGGATTTTAAAAATTTCGCGGGTC
CTAGGTGTGAGCCTTGGCGATTAGTTTACAACACGGCGTTCACAACCGATTGATACTTGA
GCCACTTGGTCAGGAATTACCAGCCCTTCACACCAGTTCCTCGACGTAGAACTAGCAGAG
CGAAGATTTCTGTCTCGAAGTGATATTGCGGTATCATTTTGGCACGTACTGTAGCTAGCG
GAGTTGCGCGAAGCGACTAGCATATACTTATCGGCGCGCTCCTGTAGCCTAAAAATGTTT
CTCAAGCCTACAGACCATTCCGAAGACAAGCTCGCAGCTAACTAGGGCCGGAAGACAAGA
CGAGCCTGAGGTAACGGTCTGCCAGTTTGGAATCAACGCTACGTGAGAAAGCGACTACCA
ACCGATTGGTTTCCCTTTTTGTCTTCGACTATGTCGGATTGACTGTTAGAGGTGCTACTT
GGAGTTGCTCATGTCCGGAACGGTCACGAAGAAGTCCTAAACGACCCTTGGGCTGTGTGT
ACAGCTCACCACGAAGAGATGTCGTCGTAGATATGCAACCCGGAACCTACCAGTACCCGT
AAACACTGCTGCCGAGCCCGGGTATGGCGCACCGCCCCCCTGGGGCTATTTCATGCCTTA
ATAGGCCCCAACTTCATAATAACTTTCGAAGAAGCTCAAAGCGCTATGCCAGGCAGTAGC
TGCCACGTGAGCCCAAAATAG

Another useful method to parse a sequence file is into a dictionary. This allows you to retrieve entries by their id field. In a FASTA file, the id is interpreted to be everything between '`>`' and the first space.

Let's retrieve 5 records of a random id between 0 and 99.

In [19]:
seq_record_dict = Bio.SeqIO.to_dict(
                            Bio.SeqIO.parse("l31_random_seqs.fna", "fasta"))

for i in random.sample(range(100), 4):
    print("# ID to retrieve:", i)
    print(seq_record_dict[str(i)])

# ID to retrieve: 0
ID: 0
Name: 0
Description: 0
Number of features: 0
Seq('TCAACTATCACGGGTGCGTCACCAAGGCCTACGAAAGATCCTCTCTGCTAAGCG...TTT', SingleLetterAlphabet())
# ID to retrieve: 97
ID: 97
Name: 97
Description: 97
Number of features: 0
Seq('ATCCGGCGACTTCATTGCATTATAAATAGTTCGACCTATAAATATGCACCAGGC...CTC', SingleLetterAlphabet())
# ID to retrieve: 67
ID: 67
Name: 67
Description: 67
Number of features: 0
Seq('CCGGTCACATTCATGACAAGTCTGAGAGGCATACTTGCAAGCTGCGAAGCAAGT...GTA', SingleLetterAlphabet())
# ID to retrieve: 52
ID: 52
Name: 52
Description: 52
Number of features: 0
Seq('CTATTACAAACGTTTAGAGACACCTAAGTTCCAGGGACGGTGAATTCATGTTAA...GAC', SingleLetterAlphabet())


## Working with [Multiple Sequence Alignments](https://en.wikipedia.org/wiki/Multiple_sequence_alignment)

A multiple sequence alignment (MSA) is a collection of biological sequences where each the set of characters in a given position across sequences are homologous. We will discuss the construction of a multiple sequence alignment in another tutorial.

MSAs are stored in many [formats](http://biopython.org/wiki/AlignIO#File_Formats). A FASTA file where each sequence is of the same length can be interpreted as a MSA. Although we did **not** run any alignment procedure, we can use the file we just generated as a toy alignment to discuss processing MSAs.

In [20]:
my_alignment = Bio.AlignIO.read("l31_random_seqs.fna", "fasta")
print(my_alignment)

SingleLetterAlphabet() alignment with 100 rows and 1000 columns
TCAACTATCACGGGTGCGTCACCAAGGCCTACGAAAGATCCTCT...TTT 0
CTCAAAAAAAGCCTGAATATATTATAACACATCGATTCAAGCAC...CTG 1
CTGGTTCCGAAGACAATTAGCTGGCTCCCAAATTCTTCGGATTC...CCG 2
AGTGAGCCGAGGGTCCGTGCCCACCTCCAGCTCGGCAAGCCGTA...ATG 3
CACTCCAGCTCTCTGGAAGGTTCCCTTACACCGAGTACGACCTA...GCG 4
CCGAGCCAACGGCTGAATGTGCCGTTTTAACACTAGGCACTAGC...CAA 5
AACGTCAACTATGACGGGCCATGCCGTCCACCAGGAACAGATAG...TCA 6
CGATGGTTTGGCGCAGGCATGCCCATCATCAGACCCGCGCCGCG...TTC 7
ATCACACGAGCGACGATCAGAGCTGTCTACGGAAATACCTGACT...AAA 8
AACTTTCGTCCCATGGTGGCTTTTCTGCGGAATCTCGATAGACT...TTG 9
ACTCCGTAATTCTCAGAGAATTTCCTGGAGGCGTAGGTTTAACG...AAG 10
ACCTCTGGGTAGGATGACTCTCCCGGATATTCATCTGGCCGGCA...TTA 11
TTCTTCCCCCAGTGGGCTATGAGGATGAATTTGGAACATTCTCG...AAC 12
CACCACTGCAGGCATACAATGCGGTACTCCCCTCAGATCATACT...TGT 13
TTATCACATAATCTCTACTGGTGTGGGTCTCACTTAAGTACATT...GTA 14
ACACCTGCCGGTGACCAAAATTACCCAGTTGGATGAGGTCCCCT...CTG 15
GTTAAGCTATACGGATCTTGCCATTTCAACGGCTGCGCCCCAGT...TCA 16
AATGACGTTCCAGCCACTACTGCAAGCT

Alignment objects have some convenient features that allow us to select a subset of sequences or positions.

Let's write the alignment in [Stockholm](https://en.wikipedia.org/wiki/Stockholm_format) format.


In [21]:
Bio.AlignIO.write(my_alignment, "l31_random_seqs.stk", "stockholm")

1

In [22]:
!cat l31_random_seqs.stk

# STOCKHOLM 1.0
#=GF SQ 100
0 TCAACTATCACGGGTGCGTCACCAAGGCCTACGAAAGATCCTCTCTGCTAAGCGGGCTTTTCATTCAAGATGTCGATGACCGTTTCATTTCAGCGTAATCCAGCATGCTTATGTGTGTAGGTCTATGGGGCAGCAGACATTGGCTCGTCAACGAGGGGGACTGGGTAAGTCCCTACTCTCTTGCGCCTACATTTGCTAAGCTTTCGTTTGGATAGTTGAATAAAAACTAAAATGGGGCAGAGCTTCGGCTAGTAGCAGGCGCCTTGATCAAGGTGCTGGATTTTAAAAATTTCGCGGGTCCTAGGTGTGAGCCTTGGCGATTAGTTTACAACACGGCGTTCACAACCGATTGATACTTGAGCCACTTGGTCAGGAATTACCAGCCCTTCACACCAGTTCCTCGACGTAGAACTAGCAGAGCGAAGATTTCTGTCTCGAAGTGATATTGCGGTATCATTTTGGCACGTACTGTAGCTAGCGGAGTTGCGCGAAGCGACTAGCATATACTTATCGGCGCGCTCCTGTAGCCTAAAAATGTTTCTCAAGCCTACAGACCATTCCGAAGACAAGCTCGCAGCTAACTAGGGCCGGAAGACAAGACGAGCCTGAGGTAACGGTCTGCCAGTTTGGAATCAACGCTACGTGAGAAAGCGACTACCAACCGATTGGTTTCCCTTTTTGTCTTCGACTATGTCGGATTGACTGTTAGAGGTGCTACTTGGAGTTGCTCATGTCCGGAACGGTCACGAAGAAGTCCTAAACGACCCTTGGGCTGTGTGTACAGCTCACCACGAAGAGATGTCGTCGTAGATATGCAACCCGGAACCTACCAGTACCCGTAAACACTGCTGCCGAGCCCGGGTATGGCGCACCGCCCCCCTGGGGCTATTTCATGCCTTAATAGGCCCCAACTTCATAATAACTTTCGAAGAAGCTCAAAGCGCTATGCCAGGCAGTAGCTGCCACGTGA

Another way of understanding aligned sequences is through sequence motifs or position weight matricies (PWM). PWMs are often visualized by the propensity of a specific character at a given position is visualized by its size. Alignments can be turned into `Sequence` *Logos* given an alignment in FASTA format [here](http://weblogo.threeplusone.com/).

The relationship between sequences in an alignment is often inferred through evolutionary reconstruction methods that produce phylogenetic trees. There are many [methods](https://en.wikipedia.org/wiki/Computational_phylogenetics) to do this that fall into 4 broad categories: distace-matrix, maximum parsimony, [maximum likelihood](http://sco.h-its.org/exelixis/software.html), and [Baysian inference](http://mrbayes.sourceforge.net/). Distance-matrix methods are the most straight-forward to compute.

## NCBI [Entrez](http://www.ncbi.nlm.nih.gov/Class/MLACourse/Original8Hour/Entrez/)

The National Library of Medicine's [National Center for Biotechnology Information (NCBI)](http://www.ncbi.nlm.nih.gov/) is the repository/clearinghouse of a large part of the digital/digitized biological information and research. While you might already be familiar with one of their databases, [PubMed](http://www.ncbi.nlm.nih.gov/pubmed/), it is just one of [many](http://www.ncbi.nlm.nih.gov/guide/all/#databases_).

NCBI integrates the databases through their Entez system (pronouced as [entree](http://static.sfdict.com/staticrep/dictaudio/E02/E0218800.mp3), listen [here](https://www.youtube.com/watch?v=BCG-M5k-gvE) if you don't believe me since you've have heard it pronounced incorrectly) and programmatic access through their [E-Utilities API](http://www.ncbi.nlm.nih.gov/books/NBK25500/).

Here we use explore Entrez a little bit and then use [EFetch](http://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch) to retrieve the most recent [*E. coli* str. K-12 substr. MG1655 genome sequence](http://www.ncbi.nlm.nih.gov/nuccore/U00096.3) and parse it for interesting qualities just scratching the surface of the capabilities of Entrez and the E-Utilities API.

In [23]:
Bio.Entrez.email = "saladi@caltech.edu"
Bio.Entrez.tool = "My-First-EUtils-Query"

As of June 1, 2010, NCBI is requiring email addresses for programmatic access. Please use a *real* email address. In case of issues or (unintentional) excessive usage, NCBI will attempt to contact you before blocking, likely via your IP/hostname/domain.

Advice from the Biopython [tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc109)

>Be sensible with your usage levels. If you plan to download lots of data, consider other options. For example, if you want easy access to all the human genes, consider fetching each chromosome by FTP as a GenBank file, and importing these into your own BioSQL database.

Last I checked, NCBI restricts users to 3 queries every second. The Entrez module has this preset, so as the user, you don't *need* to make this explicit in your code. If you find that yourself running into obscure and/or unreliably reproducable errors, try increasing the time between queries. See the `sleep()` function in the [`time` module](https://docs.python.org/2/library/time.html#time.sleep) of the standard library.

## EInfo to retrieve a list of databases

Before we go into the quering and process, let's appreciate the process of querying Entrez and how long a query can take.  We will use the `timeit` module from the standard library, which we have already used in the bootcamp, since the IPython magic function `%timeit` uses it under the hood.

In [24]:
for i in range(10):
    execution_time = timeit.timeit('handle = Entrez.einfo()', 
                                   setup='from Bio import Entrez',
                                   number=1)
    # since biopython enforces a 3/second delay, pause between queries
    time.sleep(1)
    print('Execution time for trial {0:d}: {1:.2f} s'.format(i, execution_time))

Execution time for trial 0: 0.71 s
Execution time for trial 1: 0.20 s
Execution time for trial 2: 0.18 s
Execution time for trial 3: 0.18 s
Execution time for trial 4: 0.18 s
Execution time for trial 5: 0.18 s
Execution time for trial 6: 0.35 s
Execution time for trial 7: 0.18 s
Execution time for trial 8: 0.18 s
Execution time for trial 9: 0.20 s


Compare your execution times with your neighbors.  How long did it take them? Significant variability?

Querying NCBI results in a communication with their server over the internet, a query being processed on their end in the context of all the other queries, and then being returned to your computer over the internet. All these operations are affected in large part by things out of your control, e.g. other traffic on the network or at NCBI.

All this should remind you to stop and think for a moment to figure out whether there is a better way to do things if you have many queries.

In [28]:
handle = Bio.Entrez.einfo()
result = handle.read()
handle.close()
print(result)

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20130322//EN" "http://eutils.ncbi.nlm.nih.gov/eutils/dtd/20130322/einfo.dtd">
<eInfoResult>
<DbList>

	<DbName>pubmed</DbName>
	<DbName>protein</DbName>
	<DbName>nuccore</DbName>
	<DbName>nucleotide</DbName>
	<DbName>nucgss</DbName>
	<DbName>nucest</DbName>
	<DbName>structure</DbName>
	<DbName>genome</DbName>
	<DbName>gpipe</DbName>
	<DbName>annotinfo</DbName>
	<DbName>assembly</DbName>
	<DbName>bioproject</DbName>
	<DbName>biosample</DbName>
	<DbName>blastdbinfo</DbName>
	<DbName>books</DbName>
	<DbName>cdd</DbName>
	<DbName>clinvar</DbName>
	<DbName>clone</DbName>
	<DbName>gap</DbName>
	<DbName>gapplus</DbName>
	<DbName>grasp</DbName>
	<DbName>dbvar</DbName>
	<DbName>epigenomics</DbName>
	<DbName>gene</DbName>
	<DbName>gds</DbName>
	<DbName>geoprofiles</DbName>
	<DbName>homologene</DbName>
	<DbName>medgen</DbName>
	<DbName>mesh</DbName>
	<DbName>ncbisearch</DbName>
	<DbName>nlmcatalog</DbName>
	<Db

By default, Entrez returns results in XML format. The Biopython Entrez module parses this into a Python-friendly format. The exact format will depend on the database queried!

In [29]:
record = Bio.Entrez.read(record)

TypeError: argument must have 'read' attribute

Whoops, Entrez returns a filehandle. Since we already read the handle into 'result' there is nothing left in handle to read in. We need to query Entrez once again:

In [30]:
handle = Bio.Entrez.einfo()
record = Bio.Entrez.read(handle)
handle.close()
print(record)

{'DbList': ['pubmed', 'protein', 'nuccore', 'nucleotide', 'nucgss', 'nucest', 'structure', 'genome', 'gpipe', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'clone', 'gap', 'gapplus', 'grasp', 'dbvar', 'epigenomics', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'probe', 'proteinclusters', 'pcassay', 'biosystems', 'pccompound', 'pcsubstance', 'pubmedhealth', 'seqannot', 'snp', 'sra', 'taxonomy', 'unigene', 'gencoll', 'gtr']}


Let's look more closely at the nucleotide database that we will use to retrieve the E. coli genome sequence. It can be referred to via `nuccore` or `nucleotide`. `nuccore` stands for "core nucleotide database" as opposed to `nucgss` for unannotated short read data and `nucest` for expressed sequence tag data.

In [31]:
handle = Bio.Entrez.einfo(db="nucleotide")
record = Bio.Entrez.read(handle)

print(record)

{'DbInfo': {'DbBuild': 'Build150915-1940m.1', 'LinkList': [{'Description': 'Link to Consensus CDS', 'DbTo': 'ccds', 'Name': 'nucleotide_ccds', 'Menu': 'ccds'}, {'Description': 'Genome record containing nucleotide sequence', 'DbTo': 'genome', 'Name': 'nucleotide_genome', 'Menu': 'Assembly to Genome'}], 'MenuName': 'Nucleotide', 'DbName': 'nuccore', 'Description': 'Core Nucleotide db', 'FieldList': [{'IsDate': 'N', 'FullName': 'All Fields', 'TermCount': '1798416645', 'SingleToken': 'N', 'IsNumerical': 'N', 'Description': 'All terms from all searchable fields', 'Name': 'ALL', 'IsHidden': 'N', 'Hierarchy': 'N'}, {'IsDate': 'N', 'FullName': 'UID', 'TermCount': '0', 'SingleToken': 'Y', 'IsNumerical': 'Y', 'Description': 'Unique number assigned to each sequence', 'Name': 'UID', 'IsHidden': 'Y', 'Hierarchy': 'N'}, {'IsDate': 'N', 'FullName': 'Filter', 'TermCount': '418', 'SingleToken': 'Y', 'IsNumerical': 'N', 'Description': 'Limits the records', 'Name': 'FILT', 'IsHidden': 'N', 'Hierarchy': '

Let's see if we can print this in a more intellible way:

In [32]:
for key, value in record['DbInfo'].items():
    print(key,"\t",value)

DbBuild 	 Build150915-1940m.1
LinkList 	 [{'Description': 'Link to Consensus CDS', 'DbTo': 'ccds', 'Name': 'nucleotide_ccds', 'Menu': 'ccds'}, {'Description': 'Genome record containing nucleotide sequence', 'DbTo': 'genome', 'Name': 'nucleotide_genome', 'Menu': 'Assembly to Genome'}]
MenuName 	 Nucleotide
DbName 	 nuccore
Description 	 Core Nucleotide db
FieldList 	 [{'IsDate': 'N', 'FullName': 'All Fields', 'TermCount': '1798416645', 'SingleToken': 'N', 'IsNumerical': 'N', 'Description': 'All terms from all searchable fields', 'Name': 'ALL', 'IsHidden': 'N', 'Hierarchy': 'N'}, {'IsDate': 'N', 'FullName': 'UID', 'TermCount': '0', 'SingleToken': 'Y', 'IsNumerical': 'Y', 'Description': 'Unique number assigned to each sequence', 'Name': 'UID', 'IsHidden': 'Y', 'Hierarchy': 'N'}, {'IsDate': 'N', 'FullName': 'Filter', 'TermCount': '418', 'SingleToken': 'Y', 'IsNumerical': 'N', 'Description': 'Limits the records', 'Name': 'FILT', 'IsHidden': 'N', 'Hierarchy': 'N'}, {'IsDate': 'N', 'FullName'

Once more...

In [33]:
for value in record['DbInfo']['FieldList']:
     print("%(Name)s, %(FullName)s, %(Description)s" % value)

ALL, All Fields, All terms from all searchable fields
UID, UID, Unique number assigned to each sequence
FILT, Filter, Limits the records
WORD, Text Word, Free text associated with record
TITL, Title, Words in definition line
KYWD, Keyword, Nonstandardized terms provided by submitter
AUTH, Author, Author(s) of publication
JOUR, Journal, Journal abbreviation of publication
VOL, Volume, Volume number of publication
ISS, Issue, Issue number of publication
PAGE, Page Number, Page number(s) of publication
ORGN, Organism, Scientific and common names of organism, and all higher levels of taxonomy
ACCN, Accession, Accession number of sequence
PACC, Primary Accession, Does not include retired secondary accessions
GENE, Gene Name, Name of gene associated with sequence
PROT, Protein Name, Name of protein associated with sequence
ECNO, EC/RN Number, EC number for enzyme or CAS registry number
PDAT, Publication Date, Date sequence added to GenBank
MDAT, Modification Date, Date of last update
SUBS, S

Right now, each of the fields might not have a lot of meaning, but let's retrive a sequence in genbank format and to put a couple in context.

I use the `ACCESSION` number to retrieve the sequence, but using the `GI` number would work just as well.

In [34]:
# Fetch the record
handle = Bio.Entrez.efetch(db="nucleotide",
                           id="U00096.3",
                           rettype="gb",
                           retmode="text")
record = handle.read()
handle.close()

# Print the first bit of the record (too big to print the whole thing)
print(record[:1000])

LOCUS       U00096               4641652 bp    DNA     circular BCT 01-AUG-2014
DEFINITION  Escherichia coli str. K-12 substr. MG1655, complete genome.
ACCESSION   U00096
VERSION     U00096.3  GI:545778205
DBLINK      BioProject: PRJNA225
            BioSample: SAMN02604091
KEYWORDS    .
SOURCE      Escherichia coli str. K-12 substr. MG1655
  ORGANISM  Escherichia coli str. K-12 substr. MG1655
            Bacteria; Proteobacteria; Gammaproteobacteria; Enterobacteriales;
            Enterobacteriaceae; Escherichia.
REFERENCE   1  (bases 1 to 4641652)
  AUTHORS   Blattner,F.R., Plunkett,G. III, Bloch,C.A., Perna,N.T., Burland,V.,
            Riley,M., Collado-Vides,J., Glasner,J.D., Rode,C.K., Mayhew,G.F.,
            Gregor,J., Davis,N.W., Kirkpatrick,H.A., Goeden,M.A., Rose,D.J.,
            Mau,B. and Shao,Y.
  TITLE     The complete genome sequence of Escherichia coli K-12
  JOURNAL   Science 277 (5331), 1453-1462 (1997)
   PUBMED   9278503
REFERENCE   2  (bases 1 to 4641652)
  AUTHO

I find it useful to retrieve the human-readable fileformat and save it to file. This is useful for double-check your processing code and also in case you need to process the same record again in the future.

In [35]:
genbank_file = open('U00096_3.gbk', 'w+')
genbank_file.write(record)
genbank_file.close()

!head U00096_3.gbk

LOCUS       U00096               4641652 bp    DNA     circular BCT 01-AUG-2014
DEFINITION  Escherichia coli str. K-12 substr. MG1655, complete genome.
ACCESSION   U00096
VERSION     U00096.3  GI:545778205
DBLINK      BioProject: PRJNA225
            BioSample: SAMN02604091
KEYWORDS    .
SOURCE      Escherichia coli str. K-12 substr. MG1655
  ORGANISM  Escherichia coli str. K-12 substr. MG1655
            Bacteria; Proteobacteria; Gammaproteobacteria; Enterobacteriales;


What happens if the database is specified incorrectly?

In [36]:
handle = Bio.Entrez.efetch(db="pubmed",
                           id="U00096.3",
                           rettype="gb",
                           retmode="text")
record = handle.read()
handle.close()
print(record)


1. Biophys Chem. 1975 Oct;3(4):323-9.

Polymer concentration dependence of the helix to random coil transition of a
charged polypeptide in aqueous salt solution.

Nitta K, Yoneyama M.

The helix to coil transition of poly(L-glutamic acid) was investigated in 0.05
and 0.005 M aqueous potassium chloride solutions by use of potentiometric
titration and circular dichroism measurement. Polymer concentration dependence of
the transition was observed in the range from 0.006 to 0.04 monomol/e in 0.005 M 
KG1 solution. The polymer concentration dependence can be interpreted by current 
theories of the transition of charged polypeptides and of titration curves of
linear weak polyelectrolytes taking the effect of polymer concentration into
consideration.

PMID: 96  [PubMed - indexed for MEDLINE]


2. Biochem Biophys Res Commun. 1975 Oct 27;66(4):1281-6.

Metal substitutions incarbonic anhydrase: a halide ion probe study.

Smith RJ, Bryant RG.

PMID: 3  [PubMed - indexed for MEDLINE]




That's right. Entrez tries it's best to parse the query id given: `U00096.3`, which it makes out to PMIDs: `96` and `3`. 

Thankfully, NCBI follows a set of [rules](http://www.ncbi.nlm.nih.gov/Sequin/acc.html) when assigning the accesion numbers for protein and nucleotide sequences. In fact, you can write a regular expression to identify whether a given accession number corresponds to a protein or nucleotide sequence.

NCBI also does not repeat GI numbers between the protein and nucleotide databases.

<!--
Talk about 
Parsing genbank entry mention different files types, swissprot
-->